In [ ]:
from pycorenlp import StanfordCoreNLP
import pandas as pd
import re
import corenlp
import spacy
import networkx as nx
import json


In [ ]:
topic = "AHAW VBD"

In [ ]:
df_topic_data = pd.read_csv(topic + ".csv",dtype=str, encoding="utf-8")

In [ ]:
def unicodetoascii(text):

    TEXT = (text.
            replace('\\n', " ").
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').#
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")").
            replace('\\xe2\\x80\\xa0', "").
            replace('\\x0c', "").
            replace('\\xef\\xac\\x81', "fi").
            replace('\\ufb01', "fi").
            # copyright symbol
            replace('\\xc2\\xa9', "").
            replace('\\xa9', "").
            replace('\\xe2\\x89\\xa5', "of").
            replace('\\xef\\xac\\x82', "fl").
            # degree symbol
            replace('\\xc2\\xb0', " ").
            replace('\\xb0', " ").
            replace('\\xe8', "").
            replace('\\', "")

    
                 )
    return TEXT

In [ ]:
def proc_sentence(tokens):
    """
    Takes as input a set of tokens from Stanford Core NLP output and returns 
    the set of peoplefound within the sentence. This relies on the fact that
    named entities which are contiguous within a sentence should be part of 
    the same name. For example, in the following:
    [
        {'word': 'Brandon', 'ner': 'PERSON'},
        {'word': 'Rose', 'ner': 'PERSON'},
        {'word': 'eats', 'ner': 'O'},
        {'word': 'bananas', 'ner': 'O'}
    ]
    we can safely assume that the contiguous PERSONs Brandon + Rose are part of the 
    same named entity, Brandon Rose.
    """
    people = set()
    token_count = 0
    for i in range(len(tokens)):
        if token_count < len(tokens):
            person = ''
            token = tokens[token_count]
            if token['ner'] == 'PERSON':
                person += token['word'].lower()
                checking = True
                while checking == True:
                    if token_count + 1 < len(tokens):
                        if tokens[token_count + 1]['ner'] == 'PERSON':
                            token_count += 1
                            person += ' {}'.format(tokens[token_count]['word'].lower())
                        else:
                            checking = False
                            token_count += 1
                    else:
                        checking = False
                        token_count += 1
            else:
                token_count += 1
            if person != '':
                people.add(person)
    return people

In [ ]:
nlp = spacy.load('en')

In [ ]:
def get_triples(tokens):
    print(tokens)
    return tokens

In [ ]:
df_topic_data.head()

In [ ]:
nlp = StanfordCoreNLP('http://localhost:9000')
text = re.sub(r'\\n', " ", df_topic_data.iloc[0]['raw_text']).strip()
text = unicodetoascii(text)
output = nlp.annotate(text, properties={
  'annotators': 'parse',
  'outputFormat': 'json'
  })

In [ ]:
for token in document:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

In [ ]:
token.ent_type_

In [ ]:
# https://spacy.io/docs/usage/processing-text
document = nlp(text)

#print('document: {0}'.format(document))

# Load spacy's dependency tree into a networkx graph
edges = []
nodes = []
for token in document:
    # FYI https://spacy.io/docs/api/token
        for child in token.children:
            if not token.is_stop and not child.is_stop and token.pos_ not in ["PUNCT", "SPACE"] and child.pos_ not in ["PUNCT", "SPACE"]:
                #edges.append(('{0}-{1}'.format(token.lower_,token.i),
                #              '{0}-{1}'.format(child.lower_,child.i)))
                nodes.append(('{0}-{1}'.format(token.lower_,token.i), {"id":token.i, "name":token.lower_, "type": token.pos_}))
                nodes.append(('{0}-{1}'.format(child.lower_,child.i),{"id":child.i, "name":child.lower_, "type": child.pos_}))
                #edges.append(('{0}-{1}'.format(token.lower_,token.i),
                #              '{0}-{1}'.format(child.lower_,child.i)))
                edges.append((token.i,child.i))

graph = nx.Graph()
graph.add_nodes_from(nodes)
graph.add_edges_from(edges)

# https://networkx.github.io/documentation/networkx-1.10/reference/algorithms.shortest_paths.html
#print(nx.shortest_path_length(graph, source='robots-0', target='awesomeness-11'))
#print(nx.shortest_path(graph, source='robots-0', target='awesomeness-11'))
#print(nx.shortest_path(graph, source='robots-0', target='agency-15'))

In [ ]:
from networkx.readwrite import json_graph
d = json_graph.node_link_data(graph) # node-link format to serialize

json.dump(d, open('force/force.json','w'))

In [ ]:
%%HTML
<iframe height=400px width=100% src='force/force.html'></iframe>

In [ ]:
client = corenlp.CoreNLPClient(annotators="tokenize ssplit depparse".split(), timeout=35000)

In [ ]:
client.is_alive()

In [ ]:
client.annotate(text)

In [ ]:
with corenlp.CoreNLPClient(annotators="parse".split(), timeout=35000) as client:
  ann = client.annotate(text)

In [ ]:
ann

In [ ]:
print(output)

In [ ]:
for sent in output['sentences']:
    triples = get_triples(sent['tokens'])
    print(triples)